In [66]:
%run -i 'gen_dataset.py'

ERROR:root:File `'gen_dataset.py'` not found.


In [69]:
!jupyter nbconvert --to script gen_datasets.ipynb

[NbConvertApp] Converting notebook gen_datasets.ipynb to script
[NbConvertApp] Writing 2300 bytes to gen_datasets.py


In [55]:
import os
import json
from os import listdir
import gen_datasets
import pandas as pd
import dateutil.parser
import metadata_funs
import xlrd
# from datetime import date
import datetime
from os.path import isfile, join
import ntpath
import uuid

Read in data dictionaries

In [56]:
def read_data_dictionary():
    ex_data_path = os.path.join(os.getcwd(),'dataset_metadata/ADRF Dataset Metadata.xlsx')
    xls = pd.ExcelFile(ex_data_path)
    sheet_to_df_map = pd.read_excel(ex_data_path, sheet_name=None)
    uniform_sheets = ['course1-datasets','course2-datasets' ,'kcmo-datasets', 'in_data_2019-datasets'
    , 'mo_data_2019-datasets', 'usda-datasets','bundesbank-rc']
    lim_excel_df_sheets  = []
    for i,v in enumerate(sheet_to_df_map):
        if v in uniform_sheets:
            lim_excel_df_sheets.append(sheet_to_df_map[v])
    return lim_excel_df_sheets      

In [57]:
def gen_data_dictionary(lim_excel_df_sheets):
    common_fields = list(set(lim_excel_df_sheets[6].columns.values) & set(lim_excel_df_sheets[5].columns.values))
    new_df_list = []
    for i in range(len(lim_excel_df_sheets)):
        a = lim_excel_df_sheets[i][common_fields]
        new_df_list.append(a)
    df = pd.concat(new_df_list).drop_duplicates()
    df = df.reset_index()
    df['dataset_id'] = ["dataset-{}".format(metadata_funs.get_hash(df['title'][_])) for _ in range(len(df.index))]
#     df['uuid'] = [str(uuid.uuid4()) for _ in range(len(df.title))]
    return df

In [58]:
lim_excel_df_sheets  = read_data_dictionary()
dd_df = gen_data_dictionary(lim_excel_df_sheets)

In [59]:
# dd_df.loc[dd_df.title == 'IRI Infoscan data']

In [60]:
dd_dict = dd_df.to_dict('records')
# dd_dict

In [61]:
# dd_dict[0].keys()

In [62]:
for i in dd_dict:
    if isinstance(i['temporal_coverage_end'], datetime.date):
        i['temporal_coverage_end'] = str(dateutil.parser.parse(str(i['temporal_coverage_end'])).date())
    if isinstance(i['temporal_coverage_start'], datetime.date):
        i['temporal_coverage_start'] = str(dateutil.parser.parse(str(i['temporal_coverage_start'])).date())

In [63]:
json.dump(dd_dict, open('./datasets.json', 'w'), indent=2)